# COMPROBANDO EL FLUJO SEGUIDO DEL MAIN.PY

In [9]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from services.login_service import login
from services.autorizacion_service import check_authorization
from services.control_entregas_service import manage_delivery_control
from services.process_autorization import process_authorization
import time
from datetime import datetime, timedelta
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys



# Agregar al inicio del script para suprimir logs no deseados
from selenium.webdriver.remote.remote_connection import LOGGER

import logging

# Configurar nivel de logging
LOGGER.setLevel(logging.WARNING)
logging.basicConfig(level=logging.WARNING)
logging.getLogger('urllib3').setLevel(logging.WARNING)

In [ ]:
numeros_autorizacion = [
    "29766906",
    "29766636" #APARECE DENTRO DE CONTROL Y REGISTRO DE ENTREGA CON DOS TECNOLOGIAS SIN ENTREGAR,
    "29766521",
    "29809259",
    "29809115",
    "29808833",
    "29808738"
]

In [5]:
# En la parte donde inicias el navegador, agregar opciones para suprimir logs:
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_argument('--log-level=3')
options.add_argument('--disable-logging')

# Iniciar navegador con las opciones
driver = webdriver.Chrome(options=options)
# Iniciar sesión una sola vez al principio
#driver = webdriver.Chrome()
wait = WebDriverWait(driver, 15)
login(driver, wait)

In [6]:
numero = "29766906"
tiempo_limite_minutos=2

inicio = datetime.now()
tiempo_limite = timedelta(minutes=tiempo_limite_minutos)

print(f"\n🔁 Consultando autorización: {numero}\n")

try:
    # Volver a la página inicial para asegurar estado consistente
    driver.get("https://conexiones.saviasaludeps.com/savia/home.faces")
    wait.until(EC.presence_of_element_located((By.TAG_NAME, "h3")))
    
    # Consultar autorización
    if not check_authorization(driver, wait, numero):
        print(f"❌ Falló consulta para {numero}")
        ##return False
        
    if datetime.now() - inicio > tiempo_limite:
        print(f"⏰ Tiempo límite excedido para {numero}")
        ##return False
        
    # Volver a cargar página antes de control de entregas
    #driver.get("https://conexiones.saviasaludeps.com/savia/home.faces")
    wait.until(EC.presence_of_element_located((By.TAG_NAME, "h3")))

    # Gestionar control de entregas
    if not manage_delivery_control(driver, wait, numero):
        print(f"❌ Falló control de entregas para {numero}")
        #return False
            
    #return True

    
except Exception as e:
    print(f"❌ Error crítico al procesar {numero}: {str(e)}")
    #return False

except Exception as e:
    print(f"❌ Error al procesar autorización {numero}: {str(e)}")
    #return False


🔁 Consultando autorización: 29766906

✅ Entraste al módulo de 'Autorizaciones'
Estado Autorizada
✅ El estado es 'Autorizada'. Se procederá con el clic en 'Ver'.
✅ Diálogo de Autorizaciones cerrado con el botón X
✅ Entraste al módulo de 'Autorizaciones'
Estado Autorizada
✅ El estado es 'Autorizada'. Se procederá con el clic en 'Ver'.
✅ Se hizo clic en 'Control y registro de entregas' correctamente.
✅ Diálogo de control cerrado con el botón X (selector específico)


In [11]:
try:
        wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//h3[a[text()='Autorizaciones']]"))).click().click()
        time.sleep(1)
except:
        pass

In [4]:
def process_authorization(driver, wait, numero, tiempo_limite_minutos=2):
    inicio = datetime.now()
    tiempo_limite = timedelta(minutes=tiempo_limite_minutos)
    
    print(f"\n🔁 Consultando autorización: {numero}\n")
    
    try:
        # Volver a la página inicial para asegurar estado consistente
        driver.get("https://conexiones.saviasaludeps.com/savia/home.faces")
        wait.until(EC.presence_of_element_located((By.TAG_NAME, "h3")))
        
        # Consultar autorización
        if not check_authorization(driver, wait, numero):
            print(f"❌ Falló consulta para {numero}")
            return False
            
        if datetime.now() - inicio > tiempo_limite:
            print(f"⏰ Tiempo límite excedido para {numero}")
            return False
            
        # Volver a cargar página antes de control de entregas
        driver.get("https://conexiones.saviasaludeps.com/savia/home.faces")
        wait.until(EC.presence_of_element_located((By.TAG_NAME, "h3")))
        
        # Gestionar control de entregas
        if not manage_delivery_control(driver, wait, numero):
            print(f"❌ Falló control de entregas para {numero}")
            return False
            
        return True
        
    except Exception as e:
        print(f"❌ Error crítico al procesar {numero}: {str(e)}")
        return False
    
    except Exception as e:
        print(f"❌ Error al procesar autorización {numero}: {str(e)}")
        return False

In [ ]:
if __name__ == "__main__":
    driver = None
    
    try:
        # En la parte donde inicias el navegador, agregar opciones para suprimir logs:
        options = webdriver.ChromeOptions()
        options.add_experimental_option('excludeSwitches', ['enable-logging'])
        options.add_argument('--log-level=3')
        options.add_argument('--disable-logging')

        # Iniciar navegador con las opciones
        driver = webdriver.Chrome(options=options)
        # Iniciar sesión una sola vez al principio
        #driver = webdriver.Chrome()
        wait = WebDriverWait(driver, 15)
        login(driver, wait)
        
        for numero in numeros_autorizacion:
            exito = process_authorization(driver, wait, numero)
            
            if not exito:
                # Si falló o se excedió el tiempo, reiniciar el navegador
                print("🔄 Reiniciando navegador...")
                driver.quit()
                time.sleep(2)
                
                driver = webdriver.Chrome()
                wait = WebDriverWait(driver, 15)
                login(driver, wait)
                
    finally:
        if driver:
            driver.quit()

Main.py completo

In [8]:
authorization_numbers = [
    "29766906",
    "29766636",
    "29766521",
    "29820829", #ANULADA, 
    "29799161", #ANULADA,
    "29809259",
    "29809115",
    "29808833",
    "29808738"

]


if __name__ == "__main__":
    driver = None
    
    try:
        # En la parte donde inicias el navegador, agregar opciones para suprimir logs:
        options = webdriver.ChromeOptions()
        options.add_experimental_option('excludeSwitches', ['enable-logging'])
        options.add_argument('--log-level=3')
        options.add_argument('--disable-logging')

        # Iniciar navegador con las opciones
        driver = webdriver.Chrome(options=options)
        # Iniciar sesión una sola vez al principio
        #driver = webdriver.Chrome()
        wait = WebDriverWait(driver, 15)
        login(driver, wait)
        
        for number in authorization_numbers:
            exito = process_authorization(driver, wait, number)
            
            if not exito:
                # Si falló o se excedió el tiempo, reiniciar el navegador
                print("🔄 Reiniciando navegador...")
                driver.quit()
                time.sleep(2)
                
                driver = webdriver.Chrome()
                wait = WebDriverWait(driver, 15)
                login(driver, wait)

                
    finally:
        if driver:
            driver.quit()


🔁 Consultando autorización: 29766906

✅ Entraste al módulo de 'Autorizaciones'
Estado Autorizada
✅ El estado es 'Autorizada'. Se procederá con el clic en 'Ver'.
✅ Diálogo de Autorizaciones cerrado con el botón X
✅ Entraste al módulo de 'Autorizaciones'
Estado Autorizada
✅ El estado es 'Autorizada'. Se procederá con el clic en 'Ver'.
✅ Se hizo clic en 'Control y registro de entregas' correctamente.
✅ Diálogo de control cerrado con el botón X (selector específico)
✅ Entraste al módulo de 'Autorizaciones'
✅ Se hizo clic en 'Control y registro de entregas' correctamente.
Tipo Entrega Sin Entrega
⚠️ El Tipo Entrega es 'Sin Entrega'. Se procederá con el clic en 'Entrega'.
✅ Se hizo clic en el botón 'Entregar' correctamente.
✅ Fecha ingresada correctamente: 12/06/2025
✅ Se cerró la ventana de 'Entregar' correctamente.
✅ Diálogo de control cerrado con el botón X (selector específico)
✅ Diálogo de control cerrado mediante JavaScript
⚙️ Revision de control de fechas de entregas para 29766906 te

# COMRPROBANDO EL ESTADO DE LAS AUTORIZACIONES EN EL RPA

In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from services.login_service import login
from services.autorizacion_service import check_authorization
from services.control_entregas_service import manage_delivery_control
import time
from datetime import datetime, timedelta
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By


# Agregar al inicio del script para suprimir logs no deseados
from selenium.webdriver.remote.remote_connection import LOGGER

In [ ]:
numeros_autorizacion = [
    #"29766906",
    #"29799161"
    #"29766636",
    #"29766521",
    #"29809259",
    #"29809115",
    #"29808833",
    #"29808738"
]

In [56]:
# En la parte donde inicias el navegador, agregar opciones para suprimir logs:
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_argument('--log-level=3')
options.add_argument('--disable-logging')

# Iniciar navegador con las opciones
driver = webdriver.Chrome(options=options)
# Iniciar sesión una sola vez al principio
#driver = webdriver.Chrome()
wait = WebDriverWait(driver, 15)
login(driver, wait)

In [18]:
numero_autorizacion = [
    #"29766906",
    #"29820829", #ANULADA, 
    #"29799161" #ANULADA,
    #"29766636",
    "29766521",
    #"29809259",
    #"29809115",
    #"29808833",
    #"29808738"
]

try:
    wait.until(EC.element_to_be_clickable(
        (By.XPATH, "//h3[a[text()='Autorizaciones']]"))).click().click()
    time.sleep(1)
except:
    pass

try:
    wait.until(EC.element_to_be_clickable(
        (By.XPATH, "//span[text()='Autorizaciones']/parent::a"))).click()
    print("✅ Entraste al módulo de 'Autorizaciones'")
except Exception as e:
    print("❌ No se pudo acceder al enlace de 'Autorizaciones':", e)
    #Return

try:
    input_busqueda = wait.until(EC.presence_of_element_located(
        (By.ID, "frmAutorizaciones:tablaRegistros:j_idt78")))
    input_busqueda.clear()
    input_busqueda.send_keys(numero_autorizacion)
except Exception as e:
    print("❌ No se pudo ingresar el número de autorización:", e)
    #return

try:
    wait.until(EC.element_to_be_clickable(
        (By.XPATH, "//span[contains(@class, 'ui-icon-refresh')]/ancestor::button"))).click()
except Exception as e:
    print("❌ No se pudo hacer clic en Refrescar:", e)
    #return

time.sleep(0.5)

try:
    # Esperar a que al menos una fila con la columna "Estado" esté visible
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//td[span[@class="ui-column-title" and text()="Estado"]]'))
    )
 
    # Obtener la primera celda con columna "Estado"
    cell = driver.find_element(By.XPATH, '(//td[span[@class="ui-column-title" and text()="Estado"]])[1]')
    full_text = cell.text.strip()
 
    # Remover el encabezado si viene incluido
    if full_text.startswith("Estado"):
        text=full_text.replace("Estado", "").strip()
    else:
        text=full_text
    print(f"Estado {text}")

    if text != "Autorizada":
            print("⚠️ El estado no es 'Autorizada'. Se detiene el proceso.")
            #return
    else:
        print("✅ El estado es 'Autorizada'. Se procederá con el clic en 'Ver'.")
        #except Exception as e:
        #    print("❌ Error al verificar el estado de la tabla:", e)
        #return
        
 
except Exception as e:
    print(f"❌ No se pudo obtener el estado de la primera fila: {e}")

✅ Entraste al módulo de 'Autorizaciones'
Estado Autorizada
✅ El estado es 'Autorizada'. Se procederá con el clic en 'Ver'.


In [49]:
print(text)

Anulada


In [ ]:
# 🔍 4.5 Verificar si el estado de la fila es 'Autorizada'
try:
    wait.until(EC.presence_of_element_located((By.ID, "frmAutorizaciones:tablaRegistros_data")))
    estado_xpath = "//*[@id='frmAutorizaciones:tablaRegistros_data']/tr/td[6]"
    #//*[@id="frmAutorizaciones:tablaRegistros_data"]/tr/td[6]
    estado_element = driver.find_element(By.XPATH, estado_xpath)
    estado_texto = estado_element.text.strip()
    print(f"🔍 Estado encontrado en la primera fila: {estado_texto}")

    if estado_texto != "Autorizada":
        print("⚠️ El estado no es 'Autorizada'. Se detiene el proceso.")
        #return
    else:
        print("✅ El estado es 'Autorizada'. Se procederá con el clic en 'Ver'.")
except Exception as e:
    print("❌ Error al verificar el estado de la tabla:", e)
    #return

In [ ]:
# 🔍 4.5 Verificar si el estado de la fila es 'Autorizada'
try:
    wait.until(EC.presence_of_element_located((By.ID, "frmAutorizaciones:tablaRegistros_data")))
    estado_xpath = "//tbody[@id='frmAutorizaciones:tablaRegistros_data']/tr[1]/td[6]"
    estado_element = driver.find_element(By.XPATH, estado_xpath)
    estado_texto = estado_element.text.strip()
    print(f"🔍 Estado encontrado en la primera fila: {estado_texto}")

    if estado_texto != "Autorizada":
        print("⚠️ El estado no es 'Autorizada'. Se detiene el proceso.")
        #return
    else:
        print("✅ El estado es 'Autorizada'. Se procederá con el clic en 'Ver'.")
except Exception as e:
    print("❌ Error al verificar el estado de la tabla:", e)
    #return

# COMPROBAR Y CAMBIAR FECHAS EN EL REGISTRO DE CONTROL Y ENTREGA EN CONEXIONES RPA

In [33]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from services.login_service import login
from services.autorizacion_service import check_authorization
from services.control_entregas_service import manage_delivery_control
from services.process_autorization import process_authorization
import time
from datetime import datetime, timedelta
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.remote.remote_connection import LOGGER
import logging

# Configurar nivel de logging
LOGGER.setLevel(logging.WARNING)
logging.basicConfig(level=logging.WARNING)
logging.getLogger('urllib3').setLevel(logging.WARNING)

In [ ]:
numeros_autorizacion = [
    #"29766906",
    #"29799161"
    #"29766636",
    #"29766521",
    #"29809259",
    #"29809115",
    #"29808833",
    #"29808738"
]

In [34]:
# En la parte donde inicias el navegador, agregar opciones para suprimir logs:
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_argument('--log-level=3')
options.add_argument('--disable-logging')

# Iniciar navegador con las opciones
driver = webdriver.Chrome(options=options)
# Iniciar sesión una sola vez al principio
#driver = webdriver.Chrome()
wait = WebDriverWait(driver, 15)
login(driver, wait)

In [35]:
from selenium.webdriver.support.expected_conditions import invisibility_of_element_located

authorization_number = "29766636"
date = "12/06/2025"

try:
    wait.until(EC.element_to_be_clickable(
        (By.XPATH, "//h3[a[text()='Autorizaciones']]"))).click().click()
    time.sleep(1)
except:
    pass

try:
    wait.until(EC.element_to_be_clickable(
        (By.XPATH, "//span[text()='Autorizaciones']/parent::a"))).click()
    print("✅ Entraste al módulo de 'Autorizaciones'")
except Exception as e:
    print("❌ No se pudo acceder al enlace de 'Autorizaciones':", e)
    #return

try:
    input_search = wait.until(EC.presence_of_element_located(
        (By.ID, "frmAutorizaciones:tablaRegistros:j_idt78")))
    input_search.clear()
    input_search.send_keys(authorization_number)
except Exception as e:
    print("❌ No se pudo ingresar el número de autorización:", e)
    #return

try:
    wait.until(EC.element_to_be_clickable(
        (By.XPATH, "//span[contains(@class, 'ui-icon-refresh')]/ancestor::button"))).click()
except Exception as e:
    print("❌ No se pudo hacer clic en Refrescar:", e)
    #return
    
    
try:
    # Esperar a que se cierre completamente el diálogo anterior
    time.sleep(2)
        
    # Diccionario para almacenar los results
    results = {
        'success': False,
        'data': None,
        'error': None,
        'filepath': None
    }

    # 1. Hacer clic en "Control y Registro de Entregas"
    try:
        control_button = wait.until(EC.element_to_be_clickable((
            By.XPATH, "//button[@title='Control y registro de entregas' and contains(@class, 'ui-button')]"
        )))
        control_button.click()
        print("✅ Se hizo clic en 'Control y registro de entregas' correctamente.")
    except Exception as e:
        error_msg = f"No se pudo hacer clic en el botón de control: {str(e)}"
        print(f"❌ {error_msg}")
        results['error'] = error_msg
        #return results

except Exception as e:
        error_msg = f"Error inesperado en gestionar_control_entregas: {str(e)}"
        print(f"❌ {error_msg}")

try:
    # Esperar a que al menos una fila con la columna "Tipo Entrega" esté visible
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//td[span[@class="ui-column-title" and text()="Tipo Entrega"]]'))
    )
 
    # Obtener la primera celda con columna "Tipo Entrega"
    cell = driver.find_element(By.XPATH, '(//td[span[@class="ui-column-title" and text()="Tipo Entrega"]])[1]')
    full_text = cell.text.strip()
 
    # Remover el encabezado si viene incluido
    if full_text.startswith("Tipo Entrega"):
        text=full_text.replace("Tipo Entrega", "").strip()
    else:
        text=full_text
        print(f"Tipo Entrega {text}")

    status = text
    if status != "Sin Entrega":
            print(" ✅ El Estado Tipo Entrega 'Autorizada'. Se detiene el proceso.")
            #return None
            #return
    elif status == "Sin Entrega":
        print("⚠️ El Tipo Entrega es 'Sin Entrega'. Se procederá con el clic en 'Entrega'.")
        pass
        #except Exception as e:
        #    print("❌ Error al verificar el status de la tabla:", e)
        #return
except Exception as e:
    print(f"❌ No se pudo obtener el status de la primera fila: {e}")
    #return False





#ACA SE ESCRIBIA EL BOTON QUE TENIA EN 'ENTREGAR'

#PARA DARLE A OTRA TECNOLOGIA Y CAMBIAR SU FECHA RESPECTIVA

for i in range(10):  # Usa un número suficientemente alto como límite
    try:
        button_id = f"frmGestionar:tablaTecnologiasGestionar:{i}:j_idt319"

        # Intentar encontrar el botón sin esperar mucho (para detectar fin de lista)
        tech_button = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.ID, button_id)))
        tech_button.click()
        print(f"✅ Se hizo clic en el botón 'Entregar' de la tecnología #{i}")

        # Esperar a que el campo de fecha aparezca y esté interactuable
        input_date = wait.until(EC.visibility_of_element_located((By.ID, "frmEntrega:fechaEntregaGestionar_input")))
        input_date.click()
        input_date.send_keys(Keys.CONTROL + "a")
        input_date.send_keys(Keys.DELETE)
        input_date.send_keys(date)
        print(f"✅ Fecha ingresada para tecnología #{i}")

        # Cerrar el diálogo
        close_btn = wait.until(EC.element_to_be_clickable((
            By.XPATH, "//div[contains(@class, 'ui-dialog-titlebar') and .//span[text()='Entregar']]//a[contains(@class, 'ui-dialog-titlebar-close')]"
        )))
        close_btn.click()
        print(f"✅ Diálogo cerrado para tecnología #{i}")

        # Esperar a que el diálogo se cierre completamente
        wait.until(EC.invisibility_of_element_located((By.ID, "frmEntrega")))
        print("🕒 Confirmado que el diálogo de entrega se ha cerrado")

        time.sleep(1)  # Pausa opcional adicional

    except TimeoutException:
        print(f"ℹ️ No se encontró tecnología #{i}, se asume que no hay más tecnologías por entregar.")
        break
    except Exception as e:
        print(f"❌ Error en tecnología #{i}: {e}")
        break  # Si hay un error inesperado, también puedes cortar el ciclo si lo prefieres














# 3. Cerrar el diálogo de Control y Registro de Entregas (versión mejorada)
try:
    # Esperar a que el diálogo esté completamente cargado
    wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "div.ui-dialog[aria-labelledby='j_idt281_title']")))
            
    # Opción 1: Intentar cerrar con el botón X (versión más específica)
    try:
        close_button = wait.until(EC.element_to_be_clickable(
            (By.CSS_SELECTOR, "div.ui-dialog[aria-labelledby='j_idt281_title'] a.ui-dialog-titlebar-close")
        ))
        close_button.click()
        print("✅ Diálogo de control cerrado con el botón X (selector específico)")
    except Exception as e:
        print(f"⚠️ No se pudo cerrar con botón X (intentando alternativa): {str(e)}")
                
    # Opción 2: Intentar con el botón Salir si existe
    #try:
    #    exit_button = wait.until(EC.element_to_be_clickable(
    #        (By.XPATH, "//div[@id='j_idt281']//button[contains(text(), 'Salir')]")
    #    ))
    #    exit_button.click()
    #    print("✅ Diálogo de control cerrado con el botón Salir")
    #except Exception as e:
    #    print(f"⚠️ No se pudo cerrar con botón Salir (intentando JavaScript): {str(e)}")
                    
    # Opción 3: Forzar cierre con JavaScript
    try:
        driver.execute_script("""
        var dialogs = document.querySelectorAll('div.ui-dialog');
        dialogs.forEach(function(dialog) {
        var title = dialog.querySelector('.ui-dialog-title');
        if(title && title.textContent.includes('Control y registro de entregas')) {
        var closeBtn = dialog.querySelector('a.ui-dialog-titlebar-close');
        if(closeBtn) closeBtn.click();
        }
        });
        """)
        print("✅ Diálogo de control cerrado mediante JavaScript")
    except Exception as js_e:
        print(f"⚠️ Fallo al cerrar con JavaScript: {str(js_e)}")
                        
except Exception as e:
    print(f"❌ Error grave al intentar cerrar el diálogo: {str(e)}")

#return



✅ Entraste al módulo de 'Autorizaciones'
✅ Se hizo clic en 'Control y registro de entregas' correctamente.
Tipo Entrega Sin Entrega
⚠️ El Tipo Entrega es 'Sin Entrega'. Se procederá con el clic en 'Entrega'.
✅ Se hizo clic en el botón 'Entregar' de la tecnología #0
✅ Fecha ingresada para tecnología #0
✅ Diálogo cerrado para tecnología #0
🕒 Confirmado que el diálogo de entrega se ha cerrado
✅ Se hizo clic en el botón 'Entregar' de la tecnología #1
✅ Fecha ingresada para tecnología #1
✅ Diálogo cerrado para tecnología #1
🕒 Confirmado que el diálogo de entrega se ha cerrado
ℹ️ No se encontró tecnología #2, se asume que no hay más tecnologías por entregar.
✅ Diálogo de control cerrado con el botón X (selector específico)
✅ Diálogo de control cerrado mediante JavaScript


In [ ]:
try:
    # Esperar a que aparezca la tabla con los botones de entrega (ajusta el tiempo si es necesario)
    delivery_button = wait.until(EC.element_to_be_clickable((
        By.XPATH, "//button[@title='Entregar' and contains(@class, 'ui-button-icon-only')]"
    )))
    delivery_button.click()
    print("✅ Se hizo clic en el botón 'Entregar' correctamente.")
except Exception as e:
    print(f"❌ No se pudo hacer clic en el botón 'Entregar': {e}")


try:
    # Esperar a que esté presente el campo de date
    input_date = wait.until(EC.presence_of_element_located((
        By.ID, "frmEntrega:fechaEntregaGestionar_input"
    )))

    # Limpiar el campo con .clear() y/o teclas especiales
    input_date.click()
    input_date.send_keys(Keys.CONTROL + "a")  # Seleccionar todo
    input_date.send_keys(Keys.DELETE)         # Borrar contenido

    # Ingresar la nueva date (en el formato que el sistema espera, por ejemplo: dd/mm/yyyy)
    desired_date = date
    input_date.send_keys(desired_date)
    print(f"✅ Fecha ingresada correctamente: {desired_date}")

except Exception as e:
    print(f"❌ No se pudo ingresar la fecha en el campo: {e}")

try:
    # Esperar a que el botón de cerrar (ícono X) esté presente y clickeable
    close_button = wait.until(EC.element_to_be_clickable((
        By.XPATH, "//div[contains(@class, 'ui-dialog-titlebar') and .//span[text()='Entregar']]//a[contains(@class, 'ui-dialog-titlebar-close')]"
    )))
    close_button.click()
    print("✅ Se cerró la ventana de 'Entregar' correctamente.")
except Exception as e:
    print(f"❌ No se pudo cerrar la ventana de 'Entregar': {e}")

# Opción 1: Intentar cerrar con el botón X (versión más específica)
    try:
        close_button = wait.until(EC.element_to_be_clickable(
            (By.CSS_SELECTOR, "div.ui-dialog[aria-labelledby='j_idt281_title'] a.ui-dialog-titlebar-close")
        ))
        close_button.click()
        print("✅ Diálogo de control cerrado con el botón X (selector específico)")
    except Exception as e:
        print(f"⚠️ No se pudo cerrar con botón X (intentando alternativa): {str(e)}")

In [ ]:
#OPCIONAL 
try:
    # Esperar a que aparezca la tabla con los botones de entrega (ajusta el tiempo si es necesario)
    entrega_button = wait.until(EC.element_to_be_clickable((
        By.XPATH, "//button[@title='Ver Entregas' and contains(@class, 'ui-button-icon-only')]"
    )))
    entrega_button.click()
    print("✅ Se hizo clic en el botón 'Ver Entregas' correctamente.")
except Exception as e:
    print(f"❌ No se pudo hacer clic en el botón 'Entregar': {e}")